In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install torchmetrics

In [4]:
import torch
from torch import optim
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from utility.step_lr import StepLR
import torch.nn as nn


In [5]:
def train(swa_start, model, swa_model, epoch, device, train_loader, optimizer, log, scheduler, swa_scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        optimizer.zero_grad()

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions, targets, smoothing=0.1)
        loss.mean().backward()

        optimizer.step()

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.get_last_lr()[0])

In [6]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets, smoothing=0.1)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [7]:
if __name__ == "__main__":

    num_epochs = 50
    model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    model_name = 'vgg-swa-75'
    model_name = model_name_path + model_name
    initialize(seed=42)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)

    model.classifier[6] = nn.Linear(4096, 2)
    model.to(device)
    model.eval()

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=75)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=0.01)

    swa_model = torch.optim.swa_utils.AveragedModel(model)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
    swa_start = 25
    swa_scheduler = torch.optim.swa_utils.SWALR(optimizer, swa_lr=0.001)

    for epoch in range(num_epochs):
        train(swa_start, model, swa_model, epoch, device, dataset.train, optimizer, log, scheduler, swa_scheduler)
        test(model, device, dataset.valid, log)
        if epoch > swa_start:
          swa_model.update_parameters(model)
          swa_scheduler.step()
        else:
          scheduler.step()

    log.flush()
    # torch.save(model.state_dict(), '../model_trained/'+model_name+'.pth')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      0.3634  │      0.58 %  ┃   1.000e-03  │       00:13  ┃      0.3227  │     74.33 %  ┃
┃           1  ┃      0.2978  │      0.72 %  ┃   9.990e-04  │       00:02  ┃      0.2862  │     79.00 %  ┃
┃           2  ┃      0.2647  │      0.76 %  ┃   9.961e-04  │       00:02  ┃      0.2558  │     84.00 %  ┃
┃           3  ┃      0.2304  │      0.81 %  ┃   9.911e-04  │       00:02  ┃      0.2341  │     84.67 %  ┃
┃           4  ┃      0.2054  │      0.82 %  ┃   9.843e-04  │       00:02  ┃      0.2171  │     86.00 %  ┃
┃           5  ┃      0.1925  │      0.83 %  ┃   9.755e-04  │       00:02  ┃      0.2043  │     85.67 %  ┃
┃           6  ┃      0.1706  │      0.86 %  ┃   9.649e-04  │       00:02  ┃      0.1949  │     86.33 %  ┃
┃           7  ┃      0.1634  │      0.86 %  ┃   9.524e-04  │       00:02  ┃      0.1870  │     86.67 %  ┃
┃           8  ┃      0.1591  │      0.87 %  ┃   9.382e-04  │       00:02  ┃      0.1819  │     87.67 %  ┃
┃           9  ┃      0.1519  │      

In [8]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [9]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)


tensor(0.9703, device='cuda:0')
tensor(0.9700, device='cuda:0')
tensor(0.9608, device='cuda:0')
tensor(0.9600, device='cuda:0')
tensor(0.9800, device='cuda:0')
